<a href="https://colab.research.google.com/github/fnovoas/sinrepco/blob/main/Sinrepco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SINREPCO
por fnovoas@unal.edu.co

Para cargar un modelo de Sinrepco ya hecho (entrenado), saltamos directamente al **paso 8**. Podemos evaluarlo en el **paso 6**.

1. Instalamos las bibliotecas necesarias: TensorFlow para el desarrollo de modelos de inteligencia artificial, OpenCV para el procesamiento de imágenes y Tesseract para el reconocimiento óptico de caracteres (OCR).

In [1]:
!pip install tensorflow opencv-python pytesseract

2. Procedemos con la recolección de datos, reunimos un conjunto de datos que contiene imágenes de vehículos, tanto contaminantes como no contaminantes, estas imágenes fueron capturadas por mí en diversas ubicaciones. Contamos con 3381 imágenes de vehículos no chimenea y 280 imágenes de vehículos chimenea.
Separamos las imágenes según si contienen vehículos que emiten humo visible o no, en dos carpetas: "chimenea" y "no_chimenea". Con estos datos entrenaremos al modelo.\
Montamos Drive para acceder a las imágenes.
**Actualización**: dado que probar añadir peso a las clases para compensar el desequilibrio en la cantidad de fotos de ambas carpetas no funcionó a la hora de intentar conseguir números mayores en la diagonal de la matriz de confusión ([*paso 6.b*](https://github.com/fnovoas/sinrepco/blob/3c774b69f4f4a391a0245498577fdaa3f9d70e65/Sinrepco.ipynb)), esta vez modificamos la carpeta "chimenea", copiando y pegando sus 280 elementos 11 veces en sí misma para obtener un total de 3360 elementos (un número cercano a los 3381 elementos de su contraparte). Naturalmente, revertiremos este cambio una vez que hayamos aumentado la cantidad de datos de vehículos chimenea (esto es, cuando les tome más fotos), y no dejaremos imágenes copiadas.

In [2]:
from google.colab import drive
import os

# Montar el Google Drive
drive.mount('/content/drive')

# Directorio base donde está la carpeta "sinrepco_fotos"
base_dir = "/content/drive/My Drive/sinrepco_fotos"

# Función para contar archivos en una carpeta
def contar_archivos_en_carpeta(carpeta):
    return len([f for f in os.listdir(carpeta) if os.path.isfile(os.path.join(carpeta, f))])

# Recorrer todas las carpetas y subcarpetas en el directorio base
for root, dirs, files in os.walk(base_dir):
    for dir_name in dirs:
        carpeta_actual = os.path.join(root, dir_name)
        num_archivos = contar_archivos_en_carpeta(carpeta_actual)
        print(f"Carpeta encontrada: {carpeta_actual} - Archivos: {num_archivos}")

Mounted at /content/drive
Carpeta encontrada: /content/drive/My Drive/sinrepco_fotos/chimenea - Archivos: 0
Carpeta encontrada: /content/drive/My Drive/sinrepco_fotos/no_chimenea - Archivos: 0
Carpeta encontrada: /content/drive/My Drive/sinrepco_fotos/train - Archivos: 0
Carpeta encontrada: /content/drive/My Drive/sinrepco_fotos/validation - Archivos: 0
Carpeta encontrada: /content/drive/My Drive/sinrepco_fotos/test - Archivos: 0
Carpeta encontrada: /content/drive/My Drive/sinrepco_fotos/train/chimenea - Archivos: 2352
Carpeta encontrada: /content/drive/My Drive/sinrepco_fotos/train/no_chimenea - Archivos: 2366
Carpeta encontrada: /content/drive/My Drive/sinrepco_fotos/validation/chimenea - Archivos: 504
Carpeta encontrada: /content/drive/My Drive/sinrepco_fotos/validation/no_chimenea - Archivos: 507
Carpeta encontrada: /content/drive/My Drive/sinrepco_fotos/test/chimenea - Archivos: 504
Carpeta encontrada: /content/drive/My Drive/sinrepco_fotos/test/no_chimenea - Archivos: 508


In [3]:
#importamos las bibliotecas necesarias
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt


3. 3.a. **(Solo hace falta ejecutar este código cada vez que incorporemos nuevos datos, al hacerlo, deben estar guardados en las carpetas originales "chimenea" y "no_chimenea" en el directorio raíz respectivamente)** Cargamos y preparamos los datos de la carpeta en donde están almacenados en Drive. Este código divide las imágenes en conjuntos de entrenamiento, validación y prueba según el porcentaje especificado: el 70% de los datos irán en entrenamiento y el 15% en cada una de las otras dos categorías. Ejecutar este código cambia la estructura de la organización de los archivos y carpetas en Drive (el cambio tarda unos minutos en reflejarse en Drive).

In [ ]:
import os
import random
import shutil
import time
# Crear directorios si no existen
base_dir = '/content/drive/My Drive/sinrepco_fotos'
source_chimenea = f'{base_dir}/chimenea'
source_no_chimenea = f'{base_dir}/no_chimenea'

train_chimenea_dir = f'{base_dir}/train/chimenea'
val_chimenea_dir = f'{base_dir}/validation/chimenea'
test_chimenea_dir = f'{base_dir}/test/chimenea'

train_no_chimenea_dir = f'{base_dir}/train/no_chimenea'
val_no_chimenea_dir = f'{base_dir}/validation/no_chimenea'
test_no_chimenea_dir = f'{base_dir}/test/no_chimenea'

os.makedirs(train_chimenea_dir, exist_ok=True)
os.makedirs(val_chimenea_dir, exist_ok=True)
os.makedirs(test_chimenea_dir, exist_ok=True)
os.makedirs(train_no_chimenea_dir, exist_ok=True)
os.makedirs(val_no_chimenea_dir, exist_ok=True)
os.makedirs(test_no_chimenea_dir, exist_ok=True)

def move_files(files, source, destination):
    for f in files:
        src_path = os.path.join(source, f)
        dest_path = os.path.join(destination, f)
        shutil.move(src_path, dest_path)
        # Verifica si el archivo se movió correctamente
        if not os.path.exists(dest_path):
            print(f"Reintentando mover: {f}")
            shutil.move(src_path, dest_path)
            # Da tiempo al sistema para procesar la operación
            time.sleep(0.1)

def move_data(SOURCE, TRAINING, VALIDATION, TEST, split_train=0.7, split_val_test=0.15):
    files = [f for f in os.listdir(SOURCE) if os.path.isfile(os.path.join(SOURCE, f))]
    print(f"Total files found in {SOURCE}: {len(files)}")  # Imprime el número total de archivos encontrados

    random.shuffle(files)

    train_size = int(len(files) * split_train)
    val_size = int(len(files) * split_val_test)

    train_files = files[:train_size]
    val_files = files[train_size:train_size + val_size]
    test_files = files[train_size + val_size:]

    print(f"Moving {len(train_files)} to {TRAINING}")  # Imprime cuántos archivos se moverán al entrenamiento
    print(f"Moving {len(val_files)} to {VALIDATION}")  # Imprime cuántos archivos se moverán a validación
    print(f"Moving {len(test_files)} to {TEST}")  # Imprime cuántos archivos se moverán a prueba

    # Mueve los archivos en lotes para evitar problemas con demasiadas operaciones a la vez
    batch_size = 100  # Ajusta este tamaño según sea necesario
    for i in range(0, len(train_files), batch_size):
        move_files(train_files[i:i+batch_size], SOURCE, TRAINING)

    for i in range(0, len(val_files), batch_size):
        move_files(val_files[i:i+batch_size], SOURCE, VALIDATION)

    for i in range(0, len(test_files), batch_size):
        move_files(test_files[i:i+batch_size], SOURCE, TEST)

# Ejecutar esta función para cada clase
move_data(source_chimenea, train_chimenea_dir, val_chimenea_dir, test_chimenea_dir)
move_data(source_no_chimenea, train_no_chimenea_dir, val_no_chimenea_dir, test_no_chimenea_dir)

Total files found in /content/drive/My Drive/sinrepco_fotos/chimenea: 3360
Moving 2352 to /content/drive/My Drive/sinrepco_fotos/train/chimenea
Moving 504 to /content/drive/My Drive/sinrepco_fotos/validation/chimenea
Moving 504 to /content/drive/My Drive/sinrepco_fotos/test/chimenea
Total files found in /content/drive/My Drive/sinrepco_fotos/no_chimenea: 3381
Moving 2366 to /content/drive/My Drive/sinrepco_fotos/train/no_chimenea
Moving 507 to /content/drive/My Drive/sinrepco_fotos/validation/no_chimenea
Moving 508 to /content/drive/My Drive/sinrepco_fotos/test/no_chimenea


3.b. Después de dividir los datos, configuramos el ImageDataGenerator para cargar las imágenes de estas nuevas carpetas:

In [4]:
#importamos las bibliotecas necesarias
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator # Import the ImageDataGenerator class
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
# Parámetros (reducir estos valores si nos quedamos sin RAM)
IMG_HEIGHT = 450
IMG_WIDTH = 600
BATCH_SIZE = 8

# Data generators
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    f'{base_dir}/train',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    f'{base_dir}/validation',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    f'{base_dir}/test',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

Found 4718 images belonging to 2 classes.
Found 1011 images belonging to 2 classes.
Found 1012 images belonging to 2 classes.


4. Ahora definimos la arquitectura de la CNN. Tomamos una estructura de red neuronal convolucional, que es adecuada para el análisis de imágenes.

In [5]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Input

# Definir la arquitectura de la CNN
model = Sequential([
    Input(shape=(450, 600, 3)),  # Especificamos la forma de entrada aquí
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compilar el modelo
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

5. **Entrenamos** el modelo. Si bien el número de épocas recomendado normalmente (el número de veces que el modelo recorre todo el conjunto de datos para entrenarse) es de 20, usamos 6 para ajustarnos con el tiempo de ejecución máximo disponible (*ejecutar este código toma alrededor de 6 horas y 22 minutos*).\
Actualización: La [primera vez](https://github.com/fnovoas/sinrepco/blob/faf332cb9eff6f27528e503c696db6909bab49a5/Sinrepco.ipynb) que ejecutamos este código, encontramos que el modelo estaba clasificando incorrectamente la mayoría de las imágenes de la clase "no_chimenea" como "chimenea"; esto se debe al desequilibrio de clases en el conjunto de datos. Para compensar la falta de fotos de vehículos chimenea, aumentamos la cantidad de datos en la carpeta "chimenea" artificialmente como explicamos en el paso 2.

In [ ]:
EPOCHS = 6

# No funcionó ajustar los pesos según la proporción de las clases, dejaremos este código comentado
# class_weight = {0: 12.0, 1: 1.0}

history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=EPOCHS,
#   class_weight=class_weight
)

Epoch 1/6


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


590/590 ━━━━━━━━━━━━━━━━━━━━ 7452s 12s/step - accuracy: 0.7444 - loss: 0.6476 - val_accuracy: 0.9426 - val_loss: 0.1716
Epoch 2/6
590/590 ━━━━━━━━━━━━━━━━━━━━ 7235s 12s/step - accuracy: 0.9604 - loss: 0.1241 - val_accuracy: 0.9822 - val_loss: 0.0712
Epoch 3/6
590/590 ━━━━━━━━━━━━━━━━━━━━ 7406s 13s/step - accuracy: 0.9868 - loss: 0.0549 - val_accuracy: 0.9782 - val_loss: 0.1722
Epoch 4/6
590/590 ━━━━━━━━━━━━━━━━━━━━ 7323s 12s/step - accuracy: 0.9877 - loss: 0.0485 - val_accuracy: 0.9713 - val_loss: 0.1381
Epoch 5/6
590/590 ━━━━━━━━━━━━━━━━━━━━ 7472s 13s/step - accuracy: 0.9914 - loss: 0.0380 - val_accuracy: 0.9674 - val_loss: 0.2500
Epoch 6/6
519/590 ━━━━━━━━━━━━━━━━━━━━ 14:08 12s/step - accuracy: 0.9976 - loss: 0.0115

6. **Evaluamos** el modelo.
6.a. Gráficas de pérdida y precisión (*accuracy*).

In [ ]:
loss, accuracy = model.evaluate(test_generator)
print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

# Graficar los resultados del entrenamiento
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

6.b. Gráfica de matriz de confusión.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

# Obtener predicciones del conjunto de prueba
y_pred = model.predict(test_generator)
y_pred = np.round(y_pred).astype(int).flatten()  # Redondear a 0 o 1

# Obtener etiquetas verdaderas
y_true = test_generator.classes

# Calcular la matriz de confusión
cm = confusion_matrix(y_true, y_pred)

# Etiquetas de las clases
class_names = ['no_chimenea', 'chimenea']

# Graficar la matriz de confusión
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

6.c. Curvas ROC y AUC.

In [ ]:
from sklearn.metrics import roc_curve, auc

# Obtener probabilidades predichas
y_pred_prob = model.predict(test_generator).flatten()

# Calcular la curva ROC
fpr, tpr, thresholds = roc_curve(y_true, y_pred_prob)
roc_auc = auc(fpr, tpr)

# Graficar la curva ROC
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

6.d. Reporte de clasificación.

In [ ]:
print(classification_report(y_true, y_pred, target_names=class_names))

6.e. Curvas de pérdida y precisión **por época**.

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

6.f. Gráfica de Precision-Recall

In [ ]:
from sklearn.metrics import precision_recall_curve

precision, recall, _ = precision_recall_curve(y_true, y_pred_prob)

plt.figure(figsize=(8, 6))
plt.plot(recall, precision, marker='.', label='Precision-Recall curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.show()

7. **Guardar** el modelo ya entrenado en Drive (para no tener que ejecutar el paso 5 nuevamente). *Tamaño del modelo actual: 2.89 GB*.

In [ ]:
# Guardar el modelo en formato Keras nativo en la carpeta sinrepco_fotos en Drive
model_path = '/content/drive/My Drive/sinrepco_fotos/saved_model.keras'

# Guardar el modelo entrenado en la ruta especificada
model.save(model_path)


7.a. Descargar el modelo en el equipo local (más demorado).

In [ ]:
# Guardar el modelo en formato Keras nativo en el almacenamiento local de Colab
model_path = '/content/saved_model.keras'

# Guardar el modelo entrenado en la ruta especificada
model.save(model_path)

# Descargar el modelo a tu equipo
from google.colab import files
files.download(model_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

8. **Cargar** el modelo a la sesión actual (para usar un modelo guardado anteriormente).

In [ ]:
from tensorflow.keras.models import load_model

# Cargar el modelo guardado
model = load_model('/content/drive/My Drive/sinrepco_fotos/saved_model.keras')

# A continuación
Deberemos desarrollar el subsistema de reconocimiento de placas vehiculares. Para un futuro avance, deberemos integrar los dos subsistemas y desarrollar la función de publicación automatizada en X (antes Twitter).
Gracias por leer.